In [27]:
##importing libraries 
import tensorflow as tf
from tensorflow.keras.models import load_model 
import pickle as pkl 
import pandas as pd 
import numpy as np

In [28]:
##Load the trained model,scaler,onehot and lable pickle files

model=load_model('models.keras')

##load the encoder and scaler 

with open('onehot_encoder_geography.pkl','rb') as file:
    onehot_encoder_geography=pkl.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pkl.load(file)
    
with open('scaler.pkl','rb') as file:
    scaler=pkl.load(file)

In [29]:
##Example Input data

input_data = {
    'CreditScore':789,
    'Gender' : 'Male',
    'Age': 45,
    'Tenure':3,
    'Balance':78000,
    'NumOfProducts':2,
    'HasCrCard': 1,
    'IsActiveMember':1,
    'EstimatedSalary':50000,
    'Geography':'France'

}

In [30]:
##one-hot encode 
geo_encode = onehot_encoder_geography.transform([[input_data['Geography']]]).toarray()
geo_enocoded_df = pd.DataFrame(geo_encode,columns=onehot_encoder_geography.get_feature_names_out(['Geography']))

d:\DeepLearning\CustomerChurn\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [31]:
##convert input data inot dataframe 
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography
0,789,Male,45,3,78000,2,1,1,50000,France


In [32]:
#Encode categorical features
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography
0,789,1,45,3,78000,2,1,1,50000,France


In [34]:
##concatination one hot encoded 
input_df = pd.concat([input_df.drop('Geography',axis=1),geo_enocoded_df],axis=1)

In [35]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,789,1,45,3,78000,2,1,1,50000,1.0,0.0,0.0


In [36]:
##scaling the input data 

input_scaled = scaler.transform(input_df)

In [37]:
input_scaled

array([[ 1.42540601,  0.91324755,  0.58015577, -0.69539349,  0.03038662,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [39]:
#predict_churn 
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


array([[0.03597045]], dtype=float32)

In [42]:
pred_prob = prediction[0][0]
pred_prob

0.035970453

In [43]:
if pred_prob > 0.5:
    print("The customer is likely to churn")
else :
    print("The customer is not likely to churn")

The customer is not likely to churn
